In [1]:
# importing required libraries
import requests
from bs4 import BeautifulSoup

import pandas as pd
import math
import re
import random 
from datetime import datetime
from collections import OrderedDict
import numpy as np

import os
import io
import tokenize
import zipfile

from radon.complexity import cc_rank, cc_visit

from tqdm.notebook import tqdm #(for Jupyter notebook)
# from tqdm import tqdm   # for all other IDEs

In [2]:
df = pd.read_json('./repo_topic.json')
df.head()

,topic,commits_date,commits_message,files_path,code_path,comment_list,docstr_list,code_metric,halstead_metric,halstead_vol,...,forks,watchers,pull_request,issues,closed_bug_issues,open_bug_issues,tags,tags_descriptor,main_branch_name,delta_time
0,/topics/3d,"[1608681600000, 1608595200000, 1595635200000, ...",[Merge pull request #39 from timgates42/bugfix...,"[SpaceshipGenerator-master/.gitignore, Spacesh...",{'SpaceshipGenerator-master/__init__.py': 'bl_...,{'SpaceshipGenerator-master/__init__.py': [' r...,{'SpaceshipGenerator-master/__init__.py': ['Pr...,{'SpaceshipGenerator-master/__init__.py': [['m...,"{'SpaceshipGenerator-master/__init__.py': [[2,...",{'SpaceshipGenerator-master/__init__.py': [15....,...,401,226,5,10.0,0,0,4,"[python, procedural-generation, blender-script...",master,1653
1,/topics/3d,"[1653955200000, 1653523200000, 1653523200000, ...","[bump version, update readme min version, Remo...","[BlenderGIS-master/.gitignore, BlenderGIS-mast...",{'BlenderGIS-master/__init__.py': '# -*- codin...,{'BlenderGIS-master/__init__.py': [' -*- codin...,{'BlenderGIS-master/__init__.py': [' Workaroun...,{'BlenderGIS-master/__init__.py': [['getAppDat...,"{'BlenderGIS-master/__init__.py': [[5, 22, 13,...",{'BlenderGIS-master/__init__.py': [166.4210625...,...,978,236,10,140.0,2,0,15,"[python, map, blender, addon, openstreetmap, g...",master,2945
2,/topics/3d,"[1656892800000, 1656892800000, 1656892800000, ...",[Merge pull request #2513 from MoritzBrueckner...,"[armory-main/.gitattributes, armory-main/.giti...",{'armory-main/Assets/ies/load_ies.py': '# Usin...,{'armory-main/Assets/ies/load_ies.py': [' Usin...,"{'armory-main/Assets/ies/load_ies.py': [], 'ar...",{'armory-main/Assets/ies/load_ies.py': [['load...,"{'armory-main/Assets/ies/load_ies.py': [[13, 6...",{'armory-main/Assets/ies/load_ies.py': [901.44...,...,266,168,2,313.0,478,177,46,"[game-engine, blender, rendering, haxe, 3d]",main,1480
3,/topics/3d,"[1656892800000, 1656892800000, 1656892800000, ...","[Bump lxml from 4.9.0 to 4.9.1 (#2936), Bump t...","[pyvista-main/.codeclimate.yml, pyvista-main/....","{'pyvista-main/setup.py': '""""""Installation fil...","{'pyvista-main/setup.py': [], 'pyvista-main/do...",{'pyvista-main/setup.py': ['Installation file ...,"{'pyvista-main/setup.py': [], 'pyvista-main/do...","{'pyvista-main/setup.py': [[0, 0, 0, 0, 0, 0, ...","{'pyvista-main/setup.py': [0], 'pyvista-main/d...",...,255,31,16,243.0,191,91,100,"[visualization, python, open-science, scientif...",main,948
4,/topics/3d,"[1639526400000, 1597968000000, 1595289600000, ...","[update readme, Update README.md, Merge pull r...","[Pixel2Mesh-master/.gitattributes, Pixel2Mesh-...",{'Pixel2Mesh-master/demo.py': '# Copyright (C...,{'Pixel2Mesh-master/demo.py': [' Copyright (C...,"{'Pixel2Mesh-master/demo.py': [], 'Pixel2Mesh-...","{'Pixel2Mesh-master/demo.py': None, 'Pixel2Mes...","{'Pixel2Mesh-master/demo.py': None, 'Pixel2Mes...","{'Pixel2Mesh-master/demo.py': [None], 'Pixel2M...",...,266,52,0,49.0,0,3,0,"[mesh, reconstruction, 3d]",master,988


In [3]:
df.columns

Index(['topic', 'commits_date', 'commits_message', 'files_path', 'code_path',
       'comment_list', 'docstr_list', 'code_metric', 'halstead_metric',
       'halstead_vol', 'cyclomatic_complexity', 'number_of_LLOC',
       'percent_comment', 'loc', 'lloc', 'sloc', 'comment', 'multi',
       'single_comments', 'blank', 'headers_list', 'partial_url', 'url',
       'stars', 'contributors', 'forks', 'watchers', 'pull_request', 'issues',
       'closed_bug_issues', 'open_bug_issues', 'tags', 'tags_descriptor',
       'main_branch_name', 'delta_time'],
      dtype='object')

In [4]:
df["score"]=(df["stars"] + df["forks"])/(1+(df["delta_time"]*df["open_bug_issues"]))#np.log(1+())
df["score"]

0     7566.000000
1     6363.000000
2        0.010513
3        0.018570
4        0.520067
5        1.491211
6     2020.000000
7      376.000000
8      445.000000
9      159.000000
10      60.000000
11      46.000000
12       0.031683
13      26.000000
14      28.000000
15       0.716628
16    2240.000000
17    1663.000000
18       0.255948
19    1101.000000
20    1098.000000
21     405.000000
22     294.000000
23     313.000000
Name: score, dtype: float64

In [5]:
df["label"]=np.where(df['score']>=.5, 'good', 'bad')
df["label"]

0     good
1     good
2      bad
3      bad
4     good
5     good
6     good
7     good
8     good
9     good
10    good
11    good
12     bad
13    good
14    good
15    good
16    good
17    good
18     bad
19    good
20    good
21    good
22    good
23    good
Name: label, dtype: object

In [6]:
df_comment_star=df.explode("comment_list")[["score", "label", "comment_list", "stars"]]
df_comment_star.head()

,score,label,comment_list,stars
0,7566.0,good,SpaceshipGenerator-master/__init__.py,7165
0,7566.0,good,SpaceshipGenerator-master/build.py,7165
0,7566.0,good,SpaceshipGenerator-master/spaceship_generator.py,7165
1,6363.0,good,BlenderGIS-master/__init__.py,5385
1,6363.0,good,BlenderGIS-master/geoscene.py,5385


In [11]:
# Dataset Class
import torch
import numpy as np
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'Good':0,
          'bad':1
         }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['label']]
        self.texts = [tokenizer(comment, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for comment in df['comment']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [12]:
# Model

from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [15]:
# Training Loop
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
            
EPOCHS = 5
model = BertClassifier()
LR = 1e-6
              
train(model, train, val, LR, EPOCHS)

In [ ]:
# evaluate
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

            test_label = test_label.to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)

            acc = (output.argmax(dim=1) == test_label).sum().item()
            total_acc_test += acc

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
evaluate(model, df_test)